In [ ]:
Таблица badges
Хранит информацию о значках, которые присуждаются за разные достижения. Например, пользователь, правильно ответивший на большое количество вопросов про PostgreSQL, может получить значок postgresql. 
Поле	Описание
id	Идентификатор значка, первичный ключ таблицы
name	Название значка
user_id	Идентификатор пользователя, которому присвоили значок, внешний ключ, отсылающий к таблице users
creation_date	Дата присвоения значка

Таблица post_types
Содержит информацию о типе постов. Их может быть два:
Question — пост с вопросом;
Answer — пост с ответом.
Поле	Описание
id	Идентификатор поста, первичный ключ таблицы
type	Тип поста

Таблица posts
Содержит информацию о постах.
Поле	Описание
id	Идентификатор поста, первичный ключ таблицы
title	Заголовок поста
creation_date	Дата создания поста
favorites_count	Число, которое показывает, сколько раз пост добавили в «Закладки»
last_activity_date	Дата последнего действия в посте, например комментария
last_edit_date	Дата последнего изменения поста
user_id	Идентификатор пользователя, который создал пост, внешний ключ к таблице users
parent_id	Если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом
post_type_id	Идентификатор типа поста, внешний ключ к таблице post_types
score	Количество очков, которое набрал пост
views_count	Количество просмотров

Таблица users
Содержит информацию о пользователях.
Поле	Описание
id	Идентификатор пользователя, первичный ключ таблицы
creation_date	Дата регистрации пользователя
display_name	Имя пользователя
last_access_date	Дата последнего входа
location	Местоположение
reputation	Очки репутации, которые получают за хорошие вопросы и полезные ответы
views	Число просмотров профиля пользователя

Таблица vote_types
Содержит информацию о типах голосов. Голос — это метка, которую пользователи ставят посту. Типов бывает несколько: 
UpMod — такую отметку получают посты с вопросами или ответами, которые пользователи посчитали уместными и полезными.
DownMod — такую отметку получают посты, которые показались пользователям наименее полезными.
Close — такую метку ставят опытные пользователи сервиса, если заданный вопрос нужно доработать или он вообще не подходит для платформы.
Offensive — такую метку могут поставить, если пользователь ответил на вопрос в грубой и оскорбительной манере, например, указав на неопытность автора поста.
Spam — такую метку ставят в случае, если пост пользователя выглядит откровенной рекламой.
Поле	Описание
id	Идентификатор типа голоса, первичный ключ
name	Название метки

Таблица votes
Содержит информацию о голосах за посты. 
Поле	Описание
id	Идентификатор голоса, первичный ключ
post_id	Идентификатор поста, внешний ключ к таблице posts
user_id	Идентификатор пользователя, который поставил посту голос, внешний ключ к таблице users
bounty_amount	Сумма вознаграждения, которое назначают, чтобы привлечь внимание к посту
vote_type_id	Идентификатор типа голоса, внешний ключ к таблице vote_types
creation_date	Дата назначения голоса


In [ ]:
1.
Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(id)
FROM stackoverflow.posts
WHERE post_type_id=1 
   AND (score>300 OR favorites_count >= 100)
GROUP BY post_type_id;

In [ ]:
2.
Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
SELECT ROUND(AVG(a.count),0)
FROM (
      SELECT COUNT(id),
             creation_date::date
      FROM stackoverflow.posts
      WHERE post_type_id = 1
      GROUP BY creation_date::date
      HAVING creation_date::date BETWEEN '2008-11-01' AND '2008-11-18') AS a;


--SELECT ROUND(AVG(a.cnt), 0)
--FROM 
--    (SELECT DATE_TRUNC('day', creation_date)::date AS dt,
--           COUNT(id) AS cnt
--    FROM stackoverflow.posts
--    WHERE post_type_id=1 
--          AND creation_date BETWEEN '2008-11-01' AND '2008-11-18'
--    GROUP BY creation_date) AS a;

In [ ]:
3.
Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT (u.id))
FROM stackoverflow.badges AS b
JOIN stackoverflow.users AS u ON b.user_id = u.id
WHERE b.creation_date::date = u.creation_date::date;

In [ ]:
4.
Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(a.id)
FROM (
     SELECT p.id
     FROM stackoverflow.posts AS p
     JOIN stackoverflow.votes AS v ON p.id = v.post_id
     JOIN stackoverflow.users AS u ON p.user_id = u.id
     WHERE u.display_name LIKE 'Joel Coehoorn'
     GROUP BY p.id
     HAVING COUNT(v.id)>=1) as a;

In [ ]:
5.
Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *,
       ROW_NUMBER() OVER(ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id;

In [ ]:
6.
Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
WITH a AS
      (SELECT v.user_id AS user_id,
             COUNT(vt.id) AS cnt
      FROM stackoverflow.votes AS v
      JOIN stackoverflow.vote_types as vt ON vt.id = v.vote_type_id
      WHERE vt.name LIKE 'Close'
      GROUP BY v.user_id
      ORDER BY cnt DESC
      LIMIT 10) 
SELECT *
FROM a
ORDER BY a.cnt DESC, a.user_id DESC;

In [ ]:
7.
Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
SELECT user_id,
       COUNT(id),
       DENSE_RANK() OVER(ORDER BY COUNT(id) DESC) AS rank
FROM stackoverflow.badges
WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY 1
ORDER BY 2 DESC, 1
LIMIT 10

In [ ]:
8.
Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
заголовок поста;
идентификатор пользователя;
число очков поста;
среднее число очков пользователя за пост, округлённое до целого числа.
Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER(PARTITION BY user_id))
FROM stackoverflow.posts
WHERE score != 0 AND title != ''

In [ ]:
9.
Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
WITH users AS
     (SELECT DISTINCT u.id,
            COUNT(b.id) AS cnt
     FROM stackoverflow.users AS u
     JOIN stackoverflow.badges AS b ON u.id = b.user_id
     GROUP BY u.id),
    posts AS
    (SELECT user_id,
            title
    FROM stackoverflow.posts
    WHERE title !='')
SELECT p.title
FROM posts AS p
JOIN users AS u ON p.user_id = u.id
WHERE cnt > 1000;

In [ ]:
10.
Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
пользователям с числом просмотров меньше 100 — группу 3.
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
SELECT DISTINCT id,
       views,
       CASE
           WHEN views >=350 THEN 1
           WHEN views >=100 AND views < 350 THEN 2
           WHEN views < 100 THEN 3
       END
FROM stackoverflow.users
WHERE location LIKE '%Canada%' AND views != 0;

In [ ]:
11.
Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH users AS    
    (SELECT DISTINCT id AS top_id,
           views,
           CASE
               WHEN views >=350 THEN 1
               WHEN views >=100 AND views < 350 THEN 2
               WHEN views < 100 THEN 3
           END AS grp
    FROM stackoverflow.users
    WHERE location LIKE '%Canada%' AND views != 0)
SELECT top.top_id,
       top.grp,
       top.views
FROM (
    SELECT top_id,
           grp,
           views,
           RANK() OVER(PARTITION BY grp ORDER BY views DESC)
    FROM users
    ORDER BY grp, views DESC) as top
WHERE rank = 1
ORDER BY 3 DESC, 1;

In [ ]:
12.
Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
номер дня;
число пользователей, зарегистрированных в этот день;
сумму пользователей с накоплением.

In [ ]:
WITH days AS
    (SELECT creation_date::date,
           COUNT(DISTINCT id) AS user_cnt,
           ROW_NUMBER() OVER() AS dnum
    FROM stackoverflow.users
    WHERE creation_date::date BETWEEN '2008-11-01' AND '2008-11-30'
    GROUP BY creation_date::date
    ORDER BY creation_date::date)
SELECT dnum,
       user_cnt,
       SUM(user_cnt) OVER(ORDER BY creation_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
FROM days;

In [ ]:
13.
Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
идентификатор пользователя;
разницу во времени между регистрацией и первым постом.

In [ ]:
WITH up AS
    (SELECT DISTINCT user_id,
           MIN(creation_date)
    FROM stackoverflow.posts
    GROUP BY user_id)
SELECT user_id,
       min - ub AS diff
FROM       
    (SELECT user_id,
           min,
           u.creation_date AS ub
    FROM up
    JOIN stackoverflow.users AS u ON up.user_id = u.id) AS crt